In [4]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Nov 24 16:05:13 2017

@author: zlchen
"""

#导入必要的库
import pandas
import numpy
import math
import csv
from pandas import read_csv

#--------------------------------数据预处理----------------------------------#
#读入数据集
Data=pandas.read_csv("MT_Train.csv")

#把文本型变量变成数字型
for name in ["job","marital","education","default","housing","loan","contact",
            "month","day_of_week","poutcome"]:
    col=pandas.Categorical(Data[name])
    Data[name]=col.codes                               #对字符串型数据进行编码
Data.head()                                            #显示对数据集编码以后的数据
#统计数据集中结果”y”列中的”yes””和”no”的个数
print(Data['y'].value_counts())

#设置随机种子
numpy.random.seed(190)

#重排数据
Data=Data.reindex(numpy.random.permutation(Data.index))
#设置训练样本占原始数据集的80%
train_max_row=math.floor(Data.shape[0]*0.8)
train=Data.iloc[:int(train_max_row)]
test=Data.iloc[int(train_max_row):]

#-------------------------随机森林模型训练-------------------------------#
from sklearn.ensemble import RandomForestClassifier
#选择x变量属性
columns=["age","job","marital","education","default","housing","loan","contact",
         "month","day_of_week","duration","campaign","pdays","previous","poutcome",
         "emp.var.rate","cons.price.idx","cons.conf.idx","euribor3m","nr.employed"]
#建立190棵树
clf=RandomForestClassifier(n_estimators=190,random_state=1,
min_samples_leaf=2)
#进行数据拟合，训练模型
clf.fit(train[columns],train["y"])
#对测试集进行预测
predictions=clf.predict(test[columns])
#输出测试集预测结果
print(predictions)

#将测试集真实结果化为一维列向量
test_y=test["y"].values.reshape(len(test["y"]),1)
from pandas import DataFrame
data1=DataFrame(test_y)

#--------------------------计算相关指标---------------------------------#
TP=0                                     #初始化真正例个数为0
T_predict=0                              #初始化预测结果为正例个数为0
T_real=0                                 #初始化样本真实结果为正例个数为0
correct=0                                #初始化样本分类正确个数为0
for i in range(1,len(data1)):
    if(predictions[i]=='yes'):
        T_predict+=1                                           #统计预测结果为正例个数      
    if(data1.iloc[i,0]=='yes'):
        T_real+=1                                              #统计样本真实结果为正例个数
    if((data1.iloc[i,0]=='yes')&(predictions[i]=='yes')):
        TP+=1                                                  #统计真正例个数
    if(data1.iloc[i,0]==predictions[i]):
        correct+=1                                             #统计样本分类正确个数
#计算并输出查准率P，召回率R，mean-F1，精度accuracy
P=(float) (TP)/T_predict
R=(float) (TP)/T_real
F1=(float) (2*P*R/(P+R))
precision=(float) (correct)/len(data1)
print(P)
print(R)
print(F1)
print(precision)

#----------------------------对未标记的测试集作出预测----------------------------#
#读入测试集
data_test=pandas.read_csv('MT_Test.csv')
#把文本型变量变成数字型
for name in ["job","marital","education","default","housing","loan","contact",
            "month","day_of_week","poutcome"]:
    col=pandas.Categorical(data_test[name])
    data_test[name]=col.codes                                  #对字符串型数据进行编码
data_test.head()                                               #显示对测试集编码以后的数据
columns=["age","job","marital","education","default","housing","loan","contact",
         "month","day_of_week","duration","campaign","pdays","previous","poutcome",
         "emp.var.rate","cons.price.idx","cons.conf.idx","euribor3m","nr.employed"]
result=clf.predict(data_test[columns])                         #对测试集进行预测
print(result)
result1=result.reshape(len(result),1)                          #将预测结果化为一维列向量
print(result1)

#-------------------------------预测结果写入csv文件------------------------------#
#调整结果至正确格式
from pandas import DataFrame
data2=DataFrame(result1)
data2.to_csv('result.csv')

df = read_csv('result.csv')
df.columns = ['SampleId', 'y']                                 #写入列名
df.to_csv('.result.csv')

with open(".result.csv","r") as source:
    rdr= csv.reader( source )
    with open("FinalResult.csv","w") as result:
       wtr= csv.writer( result )
       for r in rdr:
          del r[0]
          wtr.writerow(r)

#将预测结果写入最终csv文件
FinalData=pandas.read_csv("FinalResult.csv")
#检查结果”y”列中的”yes””和”no”的个数
FinalData['y'].value_counts()

no     8690
yes    1888
Name: y, dtype: int64
['no' 'no' 'no' ... 'no' 'no' 'yes']
0.6979166666666666
0.7127659574468085
0.7052631578947368
0.8936672967863895
['no' 'no' 'no' ... 'yes' 'no' 'yes']
[['no']
 ['no']
 ['no']
 ...
 ['yes']
 ['no']
 ['yes']]


no     13232
yes     2697
Name: y, dtype: int64